# GPT에 인터넷 검색 기능 추가하기
- DuckDuckGo 검색

## 모델 선언 및 질문
- 인터넷 검색 엔진이 없을 때의 답변 확인

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
model.invoke("2025년 현대자동차 미국 시장 전망은 어떻게 되나요?")

/Users/euni/miniforge3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content='2025년 현대자동차의 미국 시장 전망에 대해서는 여러 요인을 고려해야 합니다. 현대자동차는 최근 몇 년간 전기차 및 친환경 차량 시장에서의 입지를 강화하고 있으며, 이는 미국 시장에서도 중요한 요소로 작용할 것입니다. \n\n1. **전기차 및 친환경 차량**: 현대는 아이오닉(IONIQ) 시리즈와 같은 전기차 라인업을 통해 북미 시장에서의 경쟁력을 강화하고 있습니다. 2025년까지 미국에서의 EV(전기차) 판매 증가가 예상되므로, 현대의 새로운 전기 모델 출시와 함께 소비자 선호에 맞춘 제품 전략이 성과를 낼 것으로 보입니다.\n\n2. **소비자 친화적 트렌드**: SUV 및 크로스오버 차량에 대한 수요가 여전히 높기 때문에, 현대는 이러한 소비자 요구에 맞춘 차량 라인업을 지속적으로 확장할 것입니다.\n\n3. **자동차 기술 발전**: 자율주행 기술, 커넥티드 카, 그리고 스마트 기술 등에서의 혁신도 현대의 경쟁력을 높이는 요소가 될 것입니다. 이를 통해 소비자들에게 더욱 매력적인 제품을 제공할 수 있습니다.\n\n4. **경쟁 환경**: 미국 시장은 많은 글로벌 자동차 제조업체들이 치열하게 경쟁하는 곳입니다. 테슬라와 포드, GM 등과의 경쟁이 심화될 것으로 예상되며, 현대에게는 이러한 경쟁에 효과적으로 대응하는 것이 중요합니다.\n\n5. **환경 정책 및 규제**: 미국 내 환경 규제가 강화됨에 따라, 친환경 차량의 수요가 증가할 가능성이 큽니다. 현대는 이러한 정책 변화에 맞춰 제품 라인업을 조정할 필요가 있습니다.\n\n종합적으로, 2025년 현대자동차의 미국 시장 전망은 긍정적일 수 있으나, 글로벌 경제 상황, 공급망 문제, 경쟁 환경 등의 변수들을 신중히 고려해야 할 것입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 424, 'prompt_tokens': 21, 'total_t

## 덕덕고 설치

In [3]:
# %pip install -U duckduckgo-search
#  pip install ddgs
# 가상환경 에러날 시 pip uninstall -y langchain langchain-core langchain-community pydantic
# pip install --upgrade langchain langchain-cli langchain-openai langchain-community

Note: you may need to restart the kernel to use updated packages.


## 덕덕고 검색 엔진 정의 및 질문 실행
- 1) 에이전트는 질문에 답변하기 위해 외부 검색 도구를 사용해야 한다고 판단
- 2) 덕덕고 검색을 실행
- 3) 검색 결과가 LLM에게 전달되어 답변 생성

In [3]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(results_separator=';\n')
docs = search.invoke("2025년 현대자동차 미국 시장 전망은 어떻게 되나요?")

print(docs)


snippet: 현대자동차 미국 앨라배마(Alabama) 공장 현대자동차 미국 앨라배마(Alabama) 공장. 그렇다면 이제 미국에서는 어디에 공장을 짓고 어떤 성과를 내고 있는지 알아볼까요?, title: 타고 : 한국 자동차 산업의 역사, 현대자동차, link: https://tago.kr/story/hyundai.htm;
snippet: 향후 전망은 어떻게 되나요 ?A: 네, 현재 2025 년 저가 모델을 공식적으로 공개하지 않았지만 출시할 것이라고 보고있습니다., title: 테슬라 3분기 실적 8%대 상승: 차량 판매량 6.4% 증가 등 전망은 ?, link: https://donaemon.com/기타/테슬라-3분기-실적/;
snippet: © 2025 Google LLC., title: 중국반응| 혼다 짝퉁 취급받던 현대자동차 , 미국 시장 진출... - YouTube, link: https://www.youtube.com/watch?v=p_uFCe1m-o8;
snippet: 수익환원법과 함께 살펴보는 2024 미국 상업용 부동산 폭락 사태. 부동산 시장 전망. Posted in., title: 10년 후 한국의 아파트 부동산 시장 전망은 어떻게 될까? - RocketFolio, link: https://rocketfolio.net/the-outlook-for-south-korean-apartment-real-estate-in-10-years/


## system 메시지로 컨텍스트 보강
- RAG의 검색 증강 역할 담당

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자의 질문에 대해 아래 context에 기반하여 답변하라.:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = question_answering_prompt | model

## 검색 결과 추출 및 메모리 저장

In [5]:
from langchain_classic.memory import ChatMessageHistory

# 채팅 메시지를 저장할 메모리 객체 생성
chat_history = ChatMessageHistory() 
# 사용자 질문을 메모리에 저장
chat_history.add_user_message("2025년 현대자동차 미국 시장 전망은 어떻게 되나요?") 

# 문서 검색하고 답변 생성
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

# 생성된 답변을 메모리에 저장
chat_history.add_ai_message(answer) 

print(answer)


content='현재 2025년 현대자동차의 미국 시장에 대한 공식적인 모델 출시 계획은 공개되지 않았지만, 저가 모델을 출시할 것이라는 예측이 있습니다. 이는 현대자동차의 시장 점유율 확대와 경쟁력 강화를 위한 전략으로 해석될 수 있습니다. 앞으로의 구체적인 성과나 변화에 대해서는 추가적인 정보가 필요할 것입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 360, 'total_tokens': 441, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CcAkVkRYUq27OUR8ILxn1hSqeZEEG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--855b1057-2765-4302-b3ae-c05a9ae71625-0' usage_metadata={'input_tokens': 360, 'output_tokens': 81, 'total_tokens': 441, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## 검색 추가 설정

In [6]:
# DuckDuckGo API wrapper를 사용하여 검색할 때 검색 매개변수를 설정하기 위한 클래스 import
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# 한국 지역("kr-kr")을 기준, 최근 일주일("w") 내의 검색 결과를 가져오도록 초기화
wrapper = DuckDuckGoSearchAPIWrapper(region="kr-kr", time="w")

In [7]:
# 검색 기능을 위한 DuckDuckGoSearchResults 초기화
search = DuckDuckGoSearchResults(
    api_wrapper=wrapper,      # 앞에서 정의한 API wrapper를 사용
    source="news",            # 뉴스 소스에서만 검색하도록 지정
    results_separator=';\n'   # 결과 항목 사이에 구분자 사용 (세미콜론과 줄바꿈)
)

In [8]:
docs = search.invoke("2025년 현대자동차 미국 시장 전망은 어떻게 되나요?")

# 검색 결과 출력
print(docs)


snippet: 현대차 최고경영자(CEO) 호세 무뇨스 사장이 지난 5일, ‘2025 리더스 토크’를 통해국내 임직원들과 소통의 자리를 가졌습니다.현대차 강남대로 사옥에서 열린 이번 리더스 토크에는 호세 무뇨스 사장,이영호 부사장, 김창환 부사장, 김혜인 부사장을 비롯해 임직원 200여 명이 참석했는데요.온라인 생중계를 통해 현장에 자리하지 못한 7,500명 이상의 국내 임직원들도 함께했습니다.호세 무뇨스 사장은 지난해 12월 올해 2월에 이어 이번이 세 번째 타운홀 미, title: Hyundai Motor Group 현대자동차, ‘2025 리더스 토크’ 통해 미래 비전 공유, link: https://www.hyundaimotorgroup.com/ko/tv/CONT0000000000192869;
snippet: 5 days ago - 현대자동차와 기아가 내년 1분기를 기점으로 미국 시장 점유율 12%대 달성을 노린다 . 관세 리스크가 해소된 가운데 고수익 차종인 하이브리드 모델을 탑재한 인기 SUV(다목적스포츠차량) ..., title: Mt 현대차·기아, 美 점유율 12% 안착 목표…하이브리드 신차 모멘텀, link: https://www.mt.co.kr/industry/2025/11/10/2025110911242153007;
snippet: 1 day ago - 현대자동차가 미국 시장에 2028년까지 4년간 36조 원을 투자 한다. 또한 2030년까지 중형 픽업트럭을 미국 시장에 선보인다. 제너럴모터스(GM), 웨이모, 아마존 등 현지 기업과의 협력도 ..., title: Asiaa [현대차 글로벌 생산 재편] ⑦ 조지아·앨라배마·멕시코 생산벨트의 실체 - 아시아에이, link: https://www.asiaa.co.kr/news/articleView.html?idxno=230341;
snippet: 4 days ago - 업계 관계자는 “ 확정이 늦어지면서 상황을 예의주시하고 있다 ”고 말했다. 현대자동차와 기아의 올해 3분기 관세 비용은 각각 1조 

### 특정 사이트 내 검색 기능

In [9]:
# ytn.co.kr 사이트에서 검색
docs = search.invoke("site:ytn.co.kr 2025년 현대자동차 미국 시장 전망은 어떻게 되나요?")
docs


"snippet: [기자] 현대자동차 미국 홈페이지에 적힌 준중형 세단, 쏘나타 2025년형의 가격은 2만 6,900달러입니다. 한미 FTA 덕분에 2.5%의 자동차 관세를 면제받아 미국에서도 가격 경쟁력을 갖춰 인기를 끌어왔습니다., title: 캠리보다 비싼 쏘나타?...한일 관세 역전에 '첩첩산중' | YTN, link: https://m.ytn.co.kr/news_view.php?key=202509162020417330&s_mcd=0102;\nsnippet: 이처럼 트럼프 2기 행정부 시대와 함께, 2025 년 미국 경제에도 큰 변화의 바람이 불어닥칠 것으로 보입니다. 지금까지 뉴욕에서 YTN 이승윤입니다. YTN 이승윤 (risungyoon@ytn.co.kr)., title: [국제] 뉴욕 타임스퀘어에 100만 인파...새해 경제 전망은 ? | YTN, link: https://www.ytn.co.kr/_ln/0104_202501011251307373;\nsnippet: 특징주부터 하나 짚어보겠습니다. 대한항공 아시아나항공 합병 작업이 사실상 마무리 단계라고 봐야 되는데요. 이쪽 주가는 어떻게 반응하고 있습니까?, title: [경제] 대한항공-아시아나 합병하면 내 주식은 어떻게 되나요 ? | YTN, link: https://ytn.co.kr/_ln/0102_202411290733539034;\nsnippet: 2025 YTN 서울투어 마라톤 대회개요. Seoul Tour marathon.Q. 출발시간은 어떻게 되나요 ? 2025 YTN 서울투어마라톤 출발시간은 오전 8시 부터 신청한 종목 상관 없이 제출한 기록증 기록에 따라 그룹을 구분하여, title: 2025 YTN 서울투어 마라톤, link: https://run.ytn.co.kr/"

## 기사 링크 가져오기

In [10]:
# 검색 결과의 링크들을 저장할 빈 리스트 초기화
links = []

# 검색 결과를 세미콜론과 줄바꿈 기준으로 분리하고, 각 결과 항목에서 링크를 추출
for doc in docs.split(";\n"):
    print(doc)  # 각 검색 결과 항목을 출력하여 확인
    link = doc.split("link:")[1].strip()  # 각 항목에서 'link:' 이후의 URL 부분만 추출
    links.append(link)  # 추출한 링크를 리스트에 추가

# 모든 링크를 출력
print(links)

snippet: [기자] 현대자동차 미국 홈페이지에 적힌 준중형 세단, 쏘나타 2025년형의 가격은 2만 6,900달러입니다. 한미 FTA 덕분에 2.5%의 자동차 관세를 면제받아 미국에서도 가격 경쟁력을 갖춰 인기를 끌어왔습니다., title: 캠리보다 비싼 쏘나타?...한일 관세 역전에 '첩첩산중' | YTN, link: https://m.ytn.co.kr/news_view.php?key=202509162020417330&s_mcd=0102
snippet: 이처럼 트럼프 2기 행정부 시대와 함께, 2025 년 미국 경제에도 큰 변화의 바람이 불어닥칠 것으로 보입니다. 지금까지 뉴욕에서 YTN 이승윤입니다. YTN 이승윤 (risungyoon@ytn.co.kr)., title: [국제] 뉴욕 타임스퀘어에 100만 인파...새해 경제 전망은 ? | YTN, link: https://www.ytn.co.kr/_ln/0104_202501011251307373
snippet: 특징주부터 하나 짚어보겠습니다. 대한항공 아시아나항공 합병 작업이 사실상 마무리 단계라고 봐야 되는데요. 이쪽 주가는 어떻게 반응하고 있습니까?, title: [경제] 대한항공-아시아나 합병하면 내 주식은 어떻게 되나요 ? | YTN, link: https://ytn.co.kr/_ln/0102_202411290733539034
snippet: 2025 YTN 서울투어 마라톤 대회개요. Seoul Tour marathon.Q. 출발시간은 어떻게 되나요 ? 2025 YTN 서울투어마라톤 출발시간은 오전 8시 부터 신청한 종목 상관 없이 제출한 기록증 기록에 따라 그룹을 구분하여, title: 2025 YTN 서울투어 마라톤, link: https://run.ytn.co.kr/
['https://m.ytn.co.kr/news_view.php?key=202509162020417330&s_mcd=0102', 'https://www.ytn.co.kr/_ln/0104_202501011251307373', 

## 웹페이지 내용을 비동기로 불러오기
- Langchain의 WebBaseLoader를 사용하여 웹 페이지의 내용을 불러오기

In [11]:
from langchain_community.document_loaders import WebBaseLoader

# WebBaseLoader 객체를 생성. 'links'는 웹 페이지의 URL 목록을 담고 있는 변수
# bs_get_text_kwargs는 BeautifulSoup의 get_text() 메소드에 전달될 추가 인자
loader = WebBaseLoader(
    web_paths=links,  # 웹 페이지의 링크 목록을 지정
    bs_get_text_kwargs={
        "strip": True  # 웹 페이지에서 텍스트를 가져올 때 앞뒤의 공백을 제거
    },
)

# 비동기로 웹 페이지의 내용을 로드하고, 각 문서를 page_contents 리스트에 추가
page_contents = []  # 각 웹 페이지의 내용을 저장할 리스트입니다.
async for doc in loader.alazy_load():
    page_contents.append(doc)  # 불러온 문서를 page_contents 리스트에 추가

# page_contents에 있는 각 웹 페이지의 내용을 출력
for content in page_contents:
    print(content)  # 웹 페이지의 내용을 출력
    print('--------------')  # 페이지 구분을 위해 구분선을 출력

USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 4/4 [00:00<00:00, 17.84it/s]

page_content='캠리보다 비싼 쏘나타?...한일 관세 역전에 '첩첩산중' | YTN전체메뉴YTN경제LIVE홈정치경제사회국제연예공유하기닫기이제 해당 작성자의 댓글 내용을확인할 수 없습니다.취소차단닫기신고하기광고 또는 스팸유언비어 및 욕설, 도배, 비방글사생활 침해 또는 명예훼손음란물취소신고닫기삭제하시겠습니까?이제 해당 댓글 내용을 확인할 수 없습니다취소삭제캠리보다 비싼 쏘나타?...한일 관세 역전에 '첩첩산중'2025.09.16 오후 08:20글자 크기 설정글자 크기 설정닫기가가가가가공유하기AD[앵커]한국과 일본의 자동차 관세가 역전되면서 미국 시장에서 팔리는 현대 쏘나타 가격이 일본 토요타 캠리보다 더 비싸질 전망입니다.관세가 15%로 낮아진 일본과 달리 한미 관세협상 후속 협의가 교착상태에 빠지면서 자동차 산업의 경쟁력 저하가 우려되고 있습니다.박기완 기자가 보도합니다.[기자]현대자동차 미국 홈페이지에 적힌 준중형 세단, 쏘나타 2025년형의 가격은 2만 6,900달러입니다.한미 FTA 덕분에 2.5%의 자동차 관세를 면제받아 미국에서도 가격 경쟁력을 갖춰 인기를 끌어왔습니다.지난 4월부터는 한국산 자동차에 품목 관세 25%가 적용되고 있는데, 일단 현대차는 모든 비용 부담을 떠안고 소비자 가격을 올리지는 않았습니다.[호세 무뇨스 / 현대자동차 사장 (지난 4월) : 우리는 항상 경쟁력을 유지할 것이며 현재 미국에서 가격을 인상할 계획은 없습니다.]대신 현대차그룹은 관세 부담을 낮추기 위해 현지 생산 비중을 확대했습니다.이런 분위기를 보여주듯 우리나라의 대미 자동차 수출은 품목관세 부과 직전인 지난 3월부터 여섯 달 연속 하락 행진을 이어갔습니다.지난 7월 말 체결한 관세 협상에서 미국은 자동차 품목 관세 15% 인하를 약속했지만,대미 투자 패키지 등을 두고 양국의 줄다리기가 이어지자 미국은 모든 협의가 끝난 뒤 자동차 관세를 낮추겠다고 미루면서 압박 수단으로 활용하고 있습니다.넉 달 넘게 관세 부담을 감당해온 현대차그룹도 이제는 가격 인상이 불가피

## Beautiful Soup을 통해 특정 영역만 가져오기

In [12]:
import requests
from bs4 import BeautifulSoup

# 주어진 URL에서 기사 텍스트를 가져오는 함수
def get_article_text(url):
    try:
        # URL에 GET 요청을 보냄
        response = requests.get(url)
        # 요청이 성공하지 못하면 예외를 발생시킴
        response.raise_for_status()
        
        # BeautifulSoup을 사용하여 HTML 내용을 파싱
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 클래스가 'story-news article'인 <article> 태그를 찾음
        article = soup.find('article', class_='story-news article')
        
        # 기사를 찾았다면 그 텍스트를 반환
        if article:
            return article.get_text(strip=True)
        else:
            try:
                if soup.find('article'):
                    return soup.find('article').get_text(strip=True)
                elif soup.find('div', id="CmAdContent"):
                    return soup.find('div', id="CmAdContent").get_text(strip=True)
            except:
                return "기사 내용을 찾을 수 없습니다."
            
    # 요청이 실패할 경우 예외 처리
    except requests.exceptions.RequestException as e:
        return f"URL을 가져오는 중 오류 발생: {e}"

In [13]:
# URL 목록의 각 링크를 반복하면서 기사 텍스트를 출력
articles = []    # 가져온 내용을 리스트에 담기 위한 변수 선언
for link in links:
    print(f"URL: {link}\n")
    article_text = get_article_text(link)
    print(f"Content:\n{article_text}")
    print("--------------------------------------------------")
    articles.append(article_text)

URL: https://m.ytn.co.kr/news_view.php?key=202509162020417330&s_mcd=0102

Content:
None
--------------------------------------------------
URL: https://www.ytn.co.kr/_ln/0104_202501011251307373

Content:
[앵커]미국 뉴욕 타임스퀘어에서는 120년 전통의 새해맞이 카운트다운 행사가 열리고 있습니다.초대형 공이 떨어지는 이른바 '볼 드롭' 행사를 보려고 100만 명 이상이 모였습니다.현장에 나가 있는 취재기자를 연결합니다. 이승윤 특파원![기자]네, 뉴욕입니다.[앵커]뉴욕 타임스퀘어는 세계적인 카운트다운 명소인데, 역시 인산인해네요?[기자]네, 그렇습니다.뉴욕 타임스퀘어 새해맞이 행사는 120년째 이어지고 있습니다.타임스퀘어 자체가 매년 5천만 명이 찾는 유명 관광지기도 하지만, 새해 전야 방문객은 평균 100만 명 이상입니다.1월 1일 자정엔 타임스퀘어의 연례 전통인 '볼 드롭'이 펼쳐집니다.원 타임스 스퀘어 건물 위 42m 높이의 기둥을 따라 2,688개의 수정 삼각형으로 장식된 6톤 무게의 크리스털 공이 떨어지며 다음 연도의 숫자를 밝히는 행사입니다.오늘 만나본 미국인들은 새해에는 세계 평화와 가족들의 행복 등 다양한 소원이 꼭 이뤄지길 바라고 있었습니다.직접 들어보시죠.[멘델 / 미국인 : 저는 2025년이 전 세계에 평화가 깃들고 선의와 친절이 넘치는 해가 되기를 바랄 뿐입니다.][앰버 / 미국인 : (새해에 소원이 무엇인가요?) 행복과 친절함, 그리고 건강입니다.][앵커]2024년 마지막 거래일 뉴욕 증시는 하락 마감했군요.새해에 미국 경제 전망은 어떻습니까?[기자]인공지능 붐과 3년 반 만의 미국 연방준비제도의 금리 인하가 시장을 견인한 가운데, 뉴욕 증시는 2024년에 역사적 고점을 기록했습니다.하지만 새해에는 금리 인하 속도 조절이 예상되는 만큼 2024년의 마지막 거래일에 3대 지수는 일

In [15]:
filtered_articles = [article for article in articles if article]
chat_history.add_message("\n".join(filtered_articles))
chat_history.add_user_message("2025년 현대자동차 미국 시장 전망은 어떻게 되나요?") 

# 문서 검색하고 답변을 생성
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

# 생성된 답변 메모리에 저장
chat_history.add_ai_message(answer) 
print(answer)

content='2025년 현대자동차의 미국 시장 전망은 긍정적으로 평가될 가능성이 있습니다. 현대자동차는 미국에서 2025년형 쏘나타를 출시하며 가격 경쟁력을 갖추기 위해 한미 FTA 덕분에 2.5%의 자동차 관세를 면제받아 판매하는 전략을 계속 유지할 것으로 보입니다. 이로 인해 시장에서의 경쟁력을 강화하고, 소비자들 사이에서 인기를 끌 가능성이 높습니다.\n\n또한, 미국 경제의 변화와 함께 현대자동차는 새로운 시장 트렌드에 맞춰 다양한 전기차 및 친환경차 모델을 선보일 가능성이 있으며, 이는 소비자들에게 매력적으로 다가올 수 있습니다. 그러나 미국 시장의 전체적인 경제 상황, 경쟁업체들의 동향, 그리고 정치적 요인들이 현대자동차의 성과에 영향을 미칠 수 있는 점은 유의해야 할 사항입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 199, 'prompt_tokens': 6851, 'total_tokens': 7050, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CcAnUfVXcW4kZKOb02hun9Rp1c40U', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--81bb1b79-5fee-4205-b